In [20]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

   lot_size  house_size  number_of_properties   type  hijri_date        date  \
0   1320.00  16000000.0                     2  تجاري  1445/10/16  2024/04/25   
1   2100.00  16000000.0                     2   سكني  1445/10/16  2024/04/25   
2   3054.00  11299800.0                     1   سكني  1445/10/16  2024/04/25   
3   3218.65   9012220.0                     1   سكني  1445/10/16  2024/04/25   
4    600.00   8500000.0                     1   سكني  1445/10/16  2024/04/25   

  reference_number neighborhood    city         state  
0       23,384,117      التعاون  الرياض  منطقة الرياض  
1       23,380,125         عرقة  الرياض  منطقة الرياض  
2       23,382,550       الصفاء  الرياض  منطقة الرياض  
3       23,382,723       الصفاء  الرياض  منطقة الرياض  
4       23,384,815       الملقا  الرياض  منطقة الرياض  


In [ ]:

df = pd.read_csv("scraped_data.csv", names=["lot_size", "house_size", "number_of_properties", "type", "hijri_date", "date", "reference_number", "neighborhood", "city", "state"], header=0)

df["lot_size"] = df["lot_size"].str.replace(",", "").astype(float)
df["house_size"] = df["house_size"].str.replace(",", "").astype(float)

df["neighborhood"] = df["neighborhood"].str.split("/").str[1]

print(df.head(5))

In [ ]:


# Load the data
data = pd.read_csv('housing_data.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Handle missing values (if any)
data.dropna(subset=['price', 'foot_square', 'date'], inplace=True)

# Log transformation to normalize price and square footage distribution
data['log_price'] = np.log1p(data['price'])
data['log_foot_square'] = np.log1p(data['foot_square'])

# Set the date as the index for easier plotting
data.set_index('date', inplace=True)

# Resample the data to get monthly average log prices and log square footage (or weekly, yearly, etc.)
monthly_log_prices = data['log_price'].resample('M').mean()
monthly_log_foot_square = data['log_foot_square'].resample('M').mean()

# Plotting the log-transformed price chart
plt.figure(figsize=(14, 7))
sns.lineplot(x=monthly_log_prices.index, y=monthly_log_prices.values, label='Log-Transformed Price')
sns.lineplot(x=monthly_log_foot_square.index, y=monthly_log_foot_square.values, label='Log-Transformed Square Footage')
plt.title('Log-Transformed Average Housing Prices and Square Footage Over Time')
plt.xlabel('Date')
plt.ylabel('Log-Transformed Values')
plt.grid(True)
plt.legend()
plt.show()

# Segmentation: Create price and square footage categories
price_bins = [0, 100000, 500000, 1000000, 5000000, 10000000]
price_labels = ['0-100K', '100K-500K', '500K-1M', '1M-5M', '5M-10M']
data['price_range'] = pd.cut(data['price'], bins=price_bins, labels=price_labels)

foot_square_bins = [0, 500, 1000, 2000, 3000, 5000, 10000]
foot_square_labels = ['0-500', '500-1000', '1000-2000', '2000-3000', '3000-5000', '5000-10000']
data['foot_square_range'] = pd.cut(data['foot_square'], bins=foot_square_bins, labels=foot_square_labels)

# Aggregating median price and square footage per segment over time
median_prices_by_segment = data.groupby([pd.Grouper(freq='M'), 'price_range'])['price'].median().unstack()
median_foot_square_by_segment = data.groupby([pd.Grouper(freq='M'), 'foot_square_range'])['foot_square'].median().unstack()

# Plotting median prices by segment over time
plt.figure(figsize=(14, 7))
median_prices_by_segment.plot(ax=plt.gca())
plt.title('Median Housing Prices by Price Segment Over Time')
plt.xlabel('Date')
plt.ylabel('Median Price')
plt.legend(title='Price Range')
plt.grid(True)
plt.show()

# Plotting median square footage by segment over time
plt.figure(figsize=(14, 7))
median_foot_square_by_segment.plot(ax=plt.gca())
plt.title('Median Square Footage by Segment Over Time')
plt.xlabel('Date')
plt.ylabel('Median Square Footage')
plt.legend(title='Square Footage Range')
plt.grid(True)
plt.show()

# Scatter plot with log-transformed axes to visualize the relationship between price and square footage
plt.figure(figsize=(14, 7))
sns.scatterplot(x='log_foot_square', y='log_price', hue='city', data=data)
plt.title('Log-Transformed Price vs. Log-Transformed Square Footage')
plt.xlabel('Log-Transformed Square Footage')
plt.ylabel('Log-Transformed Price')
plt.grid(True)
plt.show()

# Heatmap to show density of deals across different ranges of price and square footage
heatmap_data = data.pivot_table(index='foot_square_range', columns='price_range', values='price', aggfunc='count')
plt.figure(figsize=(14, 7))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap='YlGnBu')
plt.title('Density of Deals Across Different Price and Square Footage Ranges')
plt.xlabel('Price Range')
plt.ylabel('Square Footage Range')
plt.show()